In [ ]:
import requests
import bs4 as bs
import pandas as pd
import Data_Manager as dm
import re

In [ ]:
def write_to_db(data):     
    if liga == 'bundesliga':
        dm.insert_bundesliga(data)
    elif liga == '2liga':
        dm.insert_liga2(data)
    elif liga == '3liga':
        dm.insert_liga3(data)
    else:
        print('liga unknown')

def read_table(table, data):
    tr = table.find_all('tr')
    for row in tr:
        td = row.find_all('td')
        
        # matchen der Teamnamen mit namen und id aus der Team tabelle
        team1_id = dm.select_team_id((td[2].a['href'].split('/')[2],))
        if team1_id == None:
            print(td[2].a['href'].split('/')[2], 'nicht gefunden')
            team1_id = (0,)
        team2_id = dm.select_team_id((td[4].a['href'].split('/')[2],))
        if team2_id == None:
            print(td[4].a['href'].split('/')[2], 'nicht gefunden')
            team2_id = (0,)

        data.append((team1_id[0], int(td[3].find_all('span')[0].text.split(':')[0]), int(td[3].find_all('span')[0].text.split(':')[1]) , team2_id[0], td[0].text))

def num_days(soup):
    options = soup.find_all('option')
    days = [option.text for option in options if option.text.find('tag') != -1]
    return len(days)

In [ ]:
liga = '3liga' # bundesliga, 2liga od. 3liga
#!!reset löscht bisherige tabelle der liga, set benutzen wenn nicht existiert
#dm.reset_spiele(liga) 
#!! für 3liga und 2liga code unten benutzen, auch set möglich
#dm.reset_spiele(liga[1:]+liga[0])
times = 0 # count requests to website
for year in range(2004,2020):
    print('Saison ', year-1, '/', year, sep = '')
    day = 1
    times +=1
    num_day = -1
    data = []
    print('Spieltag ', day, '.', sep = '', end=' ')
    
    while True:
        url = "https://www.fussballdaten.de/" + liga + "/" + str(year) + "/" + str(day) + "/"
        website = requests.get(url) 
        if website.status_code != 200:
            print('website not found', url)
            break
        soup = bs.BeautifulSoup(website.text, 'lxml')
        table = soup.table
        read_table(table, data) # appends on data
        
        if  num_day == -1:
            num_day = num_days(soup)
        elif day == num_day:
            break
        
        day += 1
        times += 1
        print(day, '.', sep = '', end=' ')
    
    write_to_db(data)

print('requests:', times)